In [ ]:
import numpy as np
import pandas as pd
import segyio
import lasio
import yaml

from pathlib import Path
from pprint import pprint

from wtie import grid, viz, autotie
from wtie.processing.logs import interpolate_nans, despike
from wtie.utils.datasets import tutorial

import matplotlib.pyplot as plt

In [ ]:
# data pathfolder = Path('.'))
seis_path_3d_path = Path(
    "c:\\Users\\Caio\\Downloads\\Rawdata\\Rawdata\\Seismic_data.sgy"
)
seis_path_2d_path = Path(
    "../data/tutorial/Poseidon/boreas1/Boreas1_seismic_alongwell_0_0.sgy"
)

las_log_3d_path = Path(
    "c:\\Users\\Caio\\Downloads\\Rawdata\\Rawdata\\Well_data\\F02-01_logs.las"
)
las_log_2d_path = Path("../data/tutorial/Poseidon/boreas1/Boreas1Decim.las")

In [ ]:
with segyio.open(seis_path_2d_path, "r") as f:
    print(f.samples.size)  # number of time samples
    print(f.ilines)
    print(f.xlines)
    print(f.offsets)  # these are actually angles, from 0 to 45 degrees
    print(f.format)
    print(f.header)  # this is the header of the file

with segyio.open(seis_path_3d_path, "r", ignore_geometry=True) as f:
    print(f.samples.size)  # number of time samples
    print(f.ilines)
    print(f.xlines)
    print(f.offsets)  # these are actually angles, from 0 to 45 degrees
    print(f.format)
    print(f.header)  # this is the header of the file
    ilines = [f.header[i][segyio.TraceField.INLINE_3D] for i in range(f.tracecount)]
    xlines = [f.header[i][segyio.TraceField.CROSSLINE_3D] for i in range(f.tracecount)]
    print(set(ilines))
    print(set(xlines))

In [ ]:
def detect_segy_format(segy_path):
    with segyio.open(segy_path, "r", ignore_geometry=True) as f:
        ilines = [f.header[i][segyio.TraceField.INLINE_3D] for i in range(f.tracecount)]
        xlines = [
            f.header[i][segyio.TraceField.CROSSLINE_3D] for i in range(f.tracecount)
        ]
        n_ilines = len(set(ilines))
        n_xlines = len(set(xlines))
        if n_ilines == 1 or n_xlines == 1:
            return "2D"
        else:
            return "3D"

In [ ]:
las_log_3d = lasio.read(las_log_3d_path)
las_log_3d = las_log_3d.df()

las_log_2d = lasio.read(las_log_2d_path)
las_log_2d = las_log_2d.df()

In [ ]:
def import_logs_generic(file_path: str) -> grid.LogSet:
    """
    Importa qualquer arquivo LAS e retorna um grid.LogSet com todos os logs disponíveis,
    mapeando nomes alternativos para 'Vp' e 'Rho' se necessário e interpolando para base regular.
    """
    file_path = Path(file_path)
    las_logs = lasio.read(file_path)
    las_df = las_logs.df()
    logs = {}
    # Define base regular (usando o menor passo encontrado)
    md = las_df.index.values
    step = np.min(np.diff(md))
    md_regular = np.arange(md[0], md[-1] + step, step)
    # Mapeamento de nomes alternativos
    name_map = {
        "Vp": [
            col
            for col in las_df.columns
            if col.startswith("DT") or col.upper() in ["VP", "DTCO", "SONIC"]
        ],
        "Rho": [
            col
            for col in las_df.columns
            if col.upper() in ["RHO", "RHOB", "RHOZ", "DENS"]
        ],
    }
    # Primeiro, tenta mapear os obrigatórios
    for key, aliases in name_map.items():
        for alias in aliases:
            if alias in las_df.columns:
                values = las_df[alias].values
                # Interpola para base regular
                interp_values = np.interp(md_regular, md, interpolate_nans(values))
                if key == "Vp":
                    if np.nanmean(values) > 100:  # provavelmente DT
                        interp_values = 1 / interp_values * 1e6 / 3.2808  # ft/us -> m/s
                logs[key] = grid.Log(interp_values, md_regular, "md", name=key)
                break
    # Adiciona os demais logs
    for col in las_df.columns:
        if col not in logs:
            try:
                values = las_df[col].values
                interp_values = np.interp(md_regular, md, interpolate_nans(values))
                logs[col] = grid.Log(interp_values, md_regular, "md", name=col)
            except Exception as e:
                print(f"Não foi possível importar o log {col}: {e}")
    return grid.LogSet(logs=logs)

In [ ]:
def import_well_path_generic(file_path: str, kb: float = 0) -> grid.WellPath:
    """
    Importa um arquivo de trajetória de poço.
    """
    file_path = Path(file_path)

    try:
        # Verifica primeiro se é formato Boreas (6 colunas)
        with open(file_path) as f:
            # Pula a primeira linha
            next(f)
            primeira_linha_dados = next(f).strip()

        # Se tem 6 números, é formato Boreas
        if len(primeira_linha_dados.split()) == 11:
            # Lê pulando apenas as duas primeiras linhas
            df = pd.read_csv(file_path, header=None, delimiter=r"\s+", skiprows=[0, 1])
            # Reorganiza os dados das 6 colunas em 3
            data = []
            for _, row in df.iterrows():
                data.extend([(row[0], row[1]), (row[4], row[6])])
            df_clean = pd.DataFrame(data, columns=["MD", "INC"])
        else:
            # Formato Volve (3 colunas)
            df = pd.read_csv(
                file_path,
                header=None,
                delimiter=r"\s+",
                comment="#",
                skip_blank_lines=True,
            )
            df.columns = ["MD", "INC", "AZI"][: len(df.columns)]
            df_clean = df[["MD", "INC"]]

        # Limpa e ordena os dados
        df_clean = df_clean.apply(pd.to_numeric, errors="coerce").dropna()
        df_clean = df_clean.sort_values("MD").drop_duplicates("MD")

        md = df_clean["MD"].values
        inc = df_clean["INC"].values

        # Para o formato Volve, garantimos que inc tem um elemento a menos que md
        if len(inc) == len(md):
            inc = inc[:-1]

        # Garante que começa em MD=0
        if md[0] != 0:
            md = np.concatenate(([0.0], md))
            inc = np.concatenate(([0.0], inc))

        # Garante que os valores são estritamente crescentes
        mask = np.diff(md) > 0
        mask = np.concatenate(([True], mask))
        md = md[mask]
        inc = inc[mask[:-1]]  # ajusta o tamanho de inc

        # Calcula TVD e converte para TVDSS
        tvd = grid.WellPath.get_tvdkb_from_inclination(md, inc)
        tvd = grid.WellPath.tvdkb_to_tvdss(tvd, kb)

        return grid.WellPath(md=md, tvdss=tvd, kb=kb)

    except Exception as e:
        raise ValueError(f"Erro ao ler arquivo {file_path}: {e}")

In [ ]:
def import_time_depth_table_generic(file_path: str) -> grid.TimeDepthTable:
    """
    Importa uma tabela tempo/profundidade genérica (espera colunas: TWT, TVDSS).
    Suporta formatos:
    1. Duas colunas simples: TWT TVDSS
    2. Formato Boreas: Depth TVDSS OWT Depth TVDSS OWT
    """
    file_path = Path(file_path)

    # Lê o arquivo ignorando comentários
    df = pd.read_csv(
        file_path, delimiter=r"\s+", header=None, comment="#", skip_blank_lines=True
    )

    # Verifica o formato pelo número de colunas
    if len(df.columns) >= 6:  # Formato Boreas
        # Reorganiza os dados das 6 colunas em 2 (TVDSS, OWT)
        data = []
        for _, row in df.iterrows():
            # Pega apenas TVDSS e OWT de cada par
            data.extend([(row[2], row[1]), (row[5], row[4])])
        df_clean = pd.DataFrame(data, columns=["TWT", "TVDSS"])
    else:  # Formato duas colunas
        df_clean = df.iloc[:, :2]
        df_clean.columns = ["TWT", "TVDSS"]

    # Limpa e ordena os dados
    df_clean = df_clean.apply(pd.to_numeric, errors="coerce").dropna()
    df_clean = df_clean.sort_values("TWT").drop_duplicates("TWT")

    # Extrai os arrays
    twt = df_clean["TWT"].values
    tvdss = df_clean["TVDSS"].values

    # OWT para TWT (se necessário)
    if np.mean(twt) < 1:  # provavelmente OWT
        twt = 2 * twt  # converte para TWT

    return grid.TimeDepthTable(twt=twt, tvdss=tvdss)

In [ ]:
def import_seismic_trace(file_path: str, trace_idx: int = 0) -> grid.Seismic:
    """
    Importa um arquivo SEGY e retorna apenas um traço específico como grid.Seismic.
    TODO: Melhorar a importação para lidar com diferentes formatos de SEGY (1D/2D/3D).
    """
    file_path = Path(file_path)
    with segyio.open(file_path, "r", ignore_geometry=True) as f:
        twt = f.samples / 1000  # ms para s
        trace = f.trace[trace_idx]  # pega o traço desejado
    return grid.Seismic(trace, twt, "twt", name=f"{file_path.stem}_trace{trace_idx}")

In [ ]:
from wtie.utils.datasets.utils import InputSet

log_path = "../data/tutorial/Poseidon/boreas1/Boreas1Decim.las"
seismic_path = "../data/tutorial/Poseidon/boreas1/Boreas1_seismic_alongwell_0_0.sgy"
wellpath_path = "../data/tutorial/Poseidon/Boreas1/Boreas1_dev.txt"
td_table_path = "../data/tutorial/Poseidon/boreas1/Boreas1_vel.txt"

logset_md = import_logs_generic(log_path)
seismic = import_seismic_trace(seismic_path)
wellpath = import_well_path_generic(wellpath_path)
td_table = import_time_depth_table_generic(td_table_path)

inputs = InputSet(
    logset_md=logset_md, seismic=seismic, wellpath=wellpath, table=td_table
)

In [ ]:
model_state_dict = Path("../data/tutorial/trained_net_state_dict.pt")
assert model_state_dict.is_file()

with open(Path("../data/tutorial/network_parameters.yaml"), "r") as yaml_file:
    training_parameters = yaml.load(yaml_file, Loader=yaml.Loader)

wavelet_extractor = tutorial.load_wavelet_extractor(
    training_parameters, model_state_dict
)

In [ ]:
modeler = tutorial.get_modeling_tool()

In [ ]:
print(autotie.tie_v1.__doc__)
print(autotie.get_default_search_space_v1.__doc__)

In [ ]:
median_length_choice = dict(
    name="logs_median_size",
    type="choice",
    values=[i for i in range(11, 63, 2)],
    value_type="int",
)

median_th_choice = dict(
    name="logs_median_threshold", type="range", bounds=[0.1, 5.5], value_type="float"
)

std_choice = dict(name="logs_std", type="range", bounds=[0.5, 5.5], value_type="float")

table_t_shift_choice = dict(
    name="table_t_shift", type="range", bounds=[-0.012, 0.012], value_type="float"
)

search_space = [
    median_length_choice,
    median_th_choice,
    std_choice,
    table_t_shift_choice,
]

In [ ]:
search_params = dict(num_iters=80, similarity_std=0.02)

In [ ]:
wavelet_scaling_params = dict(
    wavelet_min_scale=50000, wavelet_max_scale=500000, num_iters=60
)

In [ ]:
import warnings

warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")


outputs = autotie.tie_v1(
    inputs,
    wavelet_extractor,
    modeler,
    wavelet_scaling_params,
    search_params=search_params,
    search_space=search_space,
    stretch_and_squeeze_params=None,
)

In [ ]:
best_parameters, values = outputs.ax_client.get_best_parameters()
means, covariances = values
print(means)
print(covariances)

In [ ]:
pprint(best_parameters)

In [ ]:
outputs.plot_optimization_objective();

In [ ]:
fig, axes = outputs.plot_wavelet(fmax=85, phi_max=15, figsize=(6, 5))
axes[0].set_xlim((-0.1, 0.1))
axes[2].set_ylim((-12.5, 12.5))
fig.tight_layout()

In [ ]:
_scale = 120000
fig, axes = outputs.plot_tie_window(wiggle_scale=_scale, figsize=(12.0, 7.5))

In [ ]:
fig, ax = viz.plot_td_table(inputs.table, plot_params=dict(label="original"))
viz.plot_td_table(outputs.table, plot_params=dict(label="modified"), fig_axes=(fig, ax))
ax.legend(loc="best")

In [ ]:
s_and_s_params = dict(window_length=0.060, max_lag=0.010)  # in seconds

outputs2 = autotie.stretch_and_squeeze(
    inputs,
    outputs,
    wavelet_extractor,
    modeler,
    wavelet_scaling_params,
    best_parameters,
    s_and_s_params,
)

In [ ]:
fig, axes = outputs2.plot_wavelet(fmax=85, phi_max=25, figsize=(6, 5))
axes[0].set_xlim((-0.1, 0.1))
axes[2].set_ylim((-6.0, 12.5))
fig.tight_layout()

In [ ]:
fig, axes = outputs2.plot_tie_window(wiggle_scale=_scale, figsize=(12.0, 7.5))
from matplotlib.ticker import MaxNLocator

axes[0].xaxis.set_major_locator(MaxNLocator(nbins=2))

In [ ]:
fig, ax = viz.plot_td_table(inputs.table, plot_params=dict(label="original"))
viz.plot_td_table(
    outputs2.table, plot_params=dict(label="modified"), fig_axes=(fig, ax)
)
ax.legend(loc="best")

In [ ]:
fig, ax = viz.plot_warping(outputs.synth_seismic, outputs.seismic, outputs2.dlags)